In [3]:
import os
import sys
import time
import logging
import json
import ast

import cv2
from PIL import Image
import numpy as np

import torch
from torch.backends import cudnn
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
# from hrnet import HRNet

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
import torch


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(34, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)        
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 128)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = torch.nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = torch.nn.BatchNorm1d(32)
        self.dropout4 = nn.Dropout(0.5)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):        
        x = x.view(-1, 34)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)
        x = self.fc5(x)
        return x
    
    
class DNN :
    def __init__(self, weights):
        self.CLASSES = {
            0: 'STAND',
            1: 'LIE',
        }
        self.device = torch.device('cuda:0')
        self.weights = weights
        
    def load(self):
        self.model = Net()
        self.model = torch.load(self.weights)
        self.model.eval()
        
    def inference(self, x):
        data_x = self._minMaxScaling(x)  
        data = torch.tensor(data_x).to(dtype=torch.float32)     
        data = data.to(self.device)
        with torch.no_grad():
            output = self.model(data)
            print('output',output.shape)
            pred0= output.max(1, keepdim=True)
            pred= output.max(1, keepdim=True)[1]
            print('pred0',pred0)
            print('pred',pred)
        return int(pred[0][0]), float(output[0][pred[0][0]])
        
#         return pred[0][0], output[0][pred[0][0]].item()
        
    def _minMaxScaling(self,data):
        scaler = MinMaxScaler()
        scaler.fit(data)
        return scaler.transform(data)   

# pose = [[ 96.080215 , 57.818184],
#  [101.181816  ,57.818184],
#  [ 97.78075   ,56.11765 ],
#  [104.582886  ,68.02139 ],
#  [119.8877    ,73.12299 ],
#  [ 84.176476  ,90.12834 ],
#  [128.39038  ,103.73262 ],
#  [ 50.165775 , 59.51872 ],
#  [159.       ,125.83958 ],
#  [ 21.256685 , 22.106953],
#  [141.99466  , 98.63102 ],
#  [ 75.6738   ,178.55615 ],
#  [106.283424 ,183.65776 ],
#  [ 50.165775 ,229.57219 ],
#  [118.187164 ,234.6738  ],
#  [ 58.66845  ,299.29413 ],
#  [141.99466  ,300.99466 ]]

#누워있음
pose = [[ 297.92807,   -128.51799  ],
  [ 356.34534    ,335.8993   ],
  [ 213.22302    ,271.6403   ],
  [ 204.46043    ,-67.179855 ],
  [   8.76259   ,-151.8849   ],
  [  58.417267  ,  14.604317 ],
  [ 321.29498   , -87.6259   ],
  [  -1.4604317 , 381.17267  ],
  [  55.496407  , 110.99281  ],
  [ 210.30217   , -26.287771 ],
  [ 385.554     , -93.46763  ],
  [ 116.83453   ,-148.96404  ],
  [ 330.05756   , 137.28058  ],
  [ 248.27339   , 370.94965  ],
  [ 242.43166   , 125.59713  ],
  [ 338.82016   , -32.129498 ],
  [ 214.68346   ,-162.10793  ]]

pose = np.array(pose)


dnn_weights = '/DATA_17/ij/test/dnn.pth'
dnn = DNN(dnn_weights) 
dnn.load()
model = dnn.model
pred_pose_class, dnn_score  = dnn.inference(pose)
print(pred_pose_class, dnn_score )

output torch.Size([1, 2])
pred0 torch.return_types.max(
values=tensor([[10.0366]], device='cuda:0'),
indices=tensor([[1]], device='cuda:0'))
pred tensor([[1]], device='cuda:0')
1 10.036630630493164


In [ ]:
print(pose_outputs, maxvals, frontbacks )

max_batch_size = 1
input = torch.randn(max_batch_size,17,2).cuda()
dnn_weights = '/DATA_17/ij/test/dnn.pth'
dnn = DNN(dnn_weights) 
dnn.load()
model = dnn.model
output = model(input)

print(output)

torch.onnx.export(
    model,                                # model being run
    input,    # model input (or a tuple for multiple inputs)
    "dnn.onnx", # where to save the model (can be a file or file-like object)
    verbose=True, 
    opset_version=11,
    input_names = ['input'],              # the model's input names
    output_names = ['output'],
    dynamic_axes = {'input' : {0 : 'batch_size'},
                    'output' : {0 : 'batch_size'}}
)   